## Data Wrangling

- fix invalid player IDs who played in games (Mintues Played != DNP)

### 2022-23 Season

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
df = pd.read_csv('../data/processed/Season(2022-23).csv')
df.head()

,Date,Name,Team,MP,FG,FGA,FG%,3P,3PA,3P%,...,PTS,GmSc,+-,TeamAbbr,GameLink,Opponent,Home,GameType,id,fpts_fanduel
0,20221018,tyrese maxey,Philadelphia 76ers,38:11,8,16,.500,2,5,.400,...,21,13.7,-6,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20002546,30.2
1,20221018,james harden,Philadelphia 76ers,37:17,9,14,.643,5,9,.556,...,35,31.9,+1,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20000544,52.1
2,20221018,joel embiid,Philadelphia 76ers,37:16,9,18,.500,1,6,.167,...,26,17.7,-13,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20000726,48.5
3,20221018,tobias harris,Philadelphia 76ers,34:14,7,14,.500,3,6,.500,...,18,13.4,-1,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20000676,29.4
4,20221018,p.j. tucker,Philadelphia 76ers,33:01,3,5,.600,0,2,.000,...,6,3.6,-6,PHI,https://www.basketball-reference.com/boxscores...,Boston Celtics,0,Season,20000856,11.8


In [11]:
invalid_id_players = df[(df['id'] == -1) & (df['MP'] != 'DNP')]
player_count = len(invalid_id_players)
print("\nNumber of players with invalid id and MP != 'DNP':", player_count)


Number of players with invalid id and MP != 'DNP': 685


In [28]:
invalid_id_players = df[(df['id'] == -1) & (df['MP'] != 'DNP')]
player_counts = invalid_id_players.groupby('Name').size()
first_game_date = invalid_id_players.groupby('Name')['Date'].min()

player_counts_df = player_counts.reset_index(name='Count').sort_values(by='Count', ascending=False)
first_game_date_df = first_game_date.reset_index(name='FirstGameDate')
player_counts_with_date_df = pd.merge(player_counts_df, first_game_date_df, on='Name')

total_row = pd.DataFrame([['Total', player_counts_with_date_df['Count'].sum(), '']], columns=['Name', 'Count', 'FirstGameDate'])
player_counts_with_date_df = pd.concat([player_counts_with_date_df, total_row], ignore_index=True)

print(player_counts_with_date_df)

                  Name  Count FirstGameDate
0      dennis schroder     83      20221118
1            kj martin     82      20221019
2          nic claxton     80      20221019
3       alperen sengun     75      20221019
4        marcus morris     68      20221020
5           kevin knox     63      20221019
6      robert williams     55      20221216
7         theo maledon     44      20221019
8           a.j. green     35      20221022
9           vit krejci     29      20221023
10         jeff dowtin     25      20221031
11         john butler     19      20221105
12      omer yurtseven     17      20230311
13  jeenathan williams      5      20230402
14      frank kaminsky      1      20230313
15     kessler edwards      1      20230306
16          taj gibson      1      20230204
17   willy hernangomez      1      20230204
18          zeke nnaji      1      20221019
19               Total    685              


In [56]:
# df[df['id'] == 20002599]
df[df['Name'] == 'marcus morris']

,Date,Name,Team,MP,FG,FGA,FG%,3P,3PA,3P%,...,PTS,GmSc,+-,TeamAbbr,GameLink,Opponent,Home,GameType,id,fpts_fanduel
425,20221020,marcus morris,Los Angeles Clippers,22:58,5,9,.556,1,3,.333,...,14,11.2,+11,LAC,https://www.basketball-reference.com/boxscores...,Los Angeles Lakers,0,Season,-1,0.0
986,20221022,marcus morris,Los Angeles Clippers,21:02,4,7,.571,1,1,1.000,...,9,7.6,+2,LAC,https://www.basketball-reference.com/boxscores...,Sacramento Kings,0,Season,-1,0.0
1190,20221023,marcus morris,Los Angeles Clippers,31:25,10,14,.714,2,5,.400,...,22,18.4,-10,LAC,https://www.basketball-reference.com/boxscores...,Phoenix Suns,1,Season,-1,0.0
2367,20221030,marcus morris,Los Angeles Clippers,27:16,5,12,.417,2,5,.400,...,12,9.2,-5,LAC,https://www.basketball-reference.com/boxscores...,New Orleans Pelicans,1,Season,-1,0.0
2736,20221031,marcus morris,Los Angeles Clippers,37:08,5,14,.357,1,7,.143,...,11,6.3,+7,LAC,https://www.basketball-reference.com/boxscores...,Houston Rockets,1,Season,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31481,20230416,marcus morris,Los Angeles Clippers,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,LAC,https://www.basketball-reference.com/boxscores...,Phoenix Suns,0,Playoffs,-1,0.0
31651,20230418,marcus morris,Los Angeles Clippers,DNP,DNP,DNP,DNP,DNP,DNP,DNP,...,DNP,DNP,DNP,LAC,https://www.basketball-reference.com/boxscores...,Phoenix Suns,0,Playoffs,-1,0.0
31821,20230420,marcus morris,Los Angeles Clippers,19:51,2,5,.400,1,3,.333,...,5,1.9,-7,LAC,https://www.basketball-reference.com/boxscores...,Phoenix Suns,1,Playoffs,-1,0.0
31955,20230422,marcus morris,Los Angeles Clippers,24:42,3,13,.231,1,8,.125,...,9,1.9,-19,LAC,https://www.basketball-reference.com/boxscores...,Phoenix Suns,1,Playoffs,-1,0.0


In [ ]:
# TODO fix IDs for these players
'''
2022-23 Season
--------------------------------------
Zeke Nnaji --> id = 20002547
willy hernangomez	--> id = 20001405
Taj Gibson --> id = 20000517
Kessler Edwards --> id = 20002711
frank kaminsky --> id = 20001396
jeenathan williams --> id = 20002948
omer yurtseven --> id = 20002599
john butler --> id = 20002898
jeff dowtin --> id = 20002786
vit krejci --> id = 20002562
a.j. green --> id = 20002945
theo maledon --> id = 20002559
robert williams --> id = 20002024
kevin knox --> id = 20001994
marcus morris --> id = 20000847
alperen sengun --> id = 20002666
nic claxton --> id = 20002292
kj martin --> id = 20002577
dennis schroder --> id = 20000606

2023-24 Season
--------------------------------------

'''